# K₇-Riemann Verification v6: The H* Cutoff Hypothesis

## Hypothesis

The GIFT formula γₙ = (H*/8) × n^(φ-1) works in the **sub-cohomological regime** where γₙ < H* = 99.

**Key observation**: γ₂₉ ≈ 98.83 ≈ H* = 99

This suggests the domain of validity is not "n < 50" but rather "γₙ < H*", i.e., zeros smaller than the total cohomological dimension of K₇.

## Tests
1. Verify error correlation with γₙ/H* ratio
2. Find optimal cutoff in terms of γₙ (not n)
3. Test if H* is the natural scale boundary

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar

# GIFT constants
H_STAR = 99
B2, B3 = 21, 77
DIM_G2 = 14
DIM_E7 = 133
DIM_E8 = 248
PHI = (1 + np.sqrt(5)) / 2

print("GIFT Constants:")
print(f"  H* = {H_STAR}")
print(f"  φ - 1 = {PHI - 1:.6f}")
print(f"  H*/8 = {H_STAR/8:.4f}")

In [ ]:
# Extended Riemann zeros (first 150 from tables)
# Source: LMFDB / Odlyzko tables
gamma_zeros = np.array([
    14.1347, 21.0220, 25.0109, 30.4249, 32.9351, 37.5862, 40.9187, 43.3271,
    48.0052, 49.7738, 52.9703, 56.4462, 59.3470, 60.8318, 65.1125, 67.0798,
    69.5464, 72.0672, 75.7047, 77.1448, 79.3374, 82.9104, 84.7355, 87.4253,
    88.8091, 92.4919, 94.6513, 95.8706, 98.8312, 101.318, 103.726, 105.447,
    107.169, 111.030, 111.875, 114.320, 116.227, 118.791, 121.370, 122.947,
    124.257, 127.517, 129.579, 131.088, 133.498, 134.757, 138.116, 139.736,
    141.124, 143.112, 146.001, 147.423, 150.054, 150.925, 153.025, 156.113,
    157.598, 158.850, 161.189, 163.031, 165.537, 167.184, 169.095, 169.912,
    173.412, 174.754, 176.441, 178.377, 179.916, 182.207, 184.874, 185.599,
    187.229, 189.416, 192.027, 193.080, 195.265, 196.876, 198.015, 201.265,
    202.494, 204.190, 205.395, 207.906, 209.577, 211.691, 213.348, 214.547,
    216.170, 219.068, 220.715, 221.431, 224.007, 224.983, 227.421, 229.337,
    231.250, 231.987, 233.693, 236.524  # γ₁₀₀
])

n_values = np.arange(1, len(gamma_zeros) + 1)
print(f"Loaded {len(gamma_zeros)} Riemann zeros")
print(f"Range: γ₁ = {gamma_zeros[0]:.2f} to γ_{len(gamma_zeros)} = {gamma_zeros[-1]:.2f}")

In [ ]:
# GIFT prediction formula
def gift_formula(n):
    return (H_STAR / 8) * n ** (PHI - 1)

# Predictions
gamma_pred = gift_formula(n_values)

# Errors
errors = (gamma_pred - gamma_zeros) / gamma_zeros * 100
abs_errors = np.abs(errors)

print("\n" + "="*70)
print("KEY ZEROS AND GIFT CONSTANTS")
print("="*70)

key_indices = [1, 2, 14, 20, 29, 45, 50, 100]
key_constants = {
    1: ('dim(G₂)', DIM_G2),
    2: ('b₂', B2),
    14: ('κ_T⁻¹', 61),
    20: ('b₃', B3),
    29: ('H*', H_STAR),
    45: ('dim(E₇)', DIM_E7),
    100: ('—', None)
}

print(f"\n{'n':>5} {'γₙ':>10} {'GIFT const':>12} {'Value':>8} {'Match %':>10}")
print("-" * 50)
for idx in key_indices:
    if idx <= len(gamma_zeros):
        gamma = gamma_zeros[idx-1]
        name, val = key_constants.get(idx, ('—', None))
        if val:
            match = abs(gamma - val) / val * 100
            print(f"{idx:>5} {gamma:>10.2f} {name:>12} {val:>8} {match:>9.2f}%")
        else:
            print(f"{idx:>5} {gamma:>10.2f} {name:>12} {'—':>8} {'—':>10}")

In [ ]:
print("\n" + "="*70)
print("TEST 1: ERROR VS γₙ/H* RATIO")
print("="*70)

# Compute ratio γₙ/H*
gamma_ratio = gamma_zeros / H_STAR

# Find the transition point
sub_H_star = gamma_zeros < H_STAR
n_transition = np.sum(sub_H_star)  # Number of zeros below H*

print(f"\nZeros below H* = 99: n = 1 to {n_transition}")
print(f"Transition zero: γ_{n_transition} = {gamma_zeros[n_transition-1]:.2f}")
print(f"First zero above H*: γ_{n_transition+1} = {gamma_zeros[n_transition]:.2f}")

# Compute errors for sub-H* and super-H* regimes
err_sub = abs_errors[sub_H_star]
err_super = abs_errors[~sub_H_star]

print(f"\n--- Error Statistics ---")
print(f"Sub-H* regime (γₙ < 99):   mean = {np.mean(err_sub):.2f}%, max = {np.max(err_sub):.2f}%")
print(f"Super-H* regime (γₙ > 99): mean = {np.mean(err_super):.2f}%, max = {np.max(err_super):.2f}%")
print(f"\nRatio of mean errors: {np.mean(err_super)/np.mean(err_sub):.1f}x worse above H*")

In [ ]:
print("\n" + "="*70)
print("TEST 2: FIND OPTIMAL CUTOFF")
print("="*70)

# Test different cutoff values
cutoffs = [50, 77, 99, 133, 150, 200]
cutoff_names = ['~γ₅', 'b₃', 'H*', 'dim(E₇)', '~γ₃₇', '~γ₅₅']

print(f"\n{'Cutoff':>10} {'GIFT name':>12} {'n range':>12} {'Mean err':>10} {'Max err':>10}")
print("-" * 60)

for cutoff, name in zip(cutoffs, cutoff_names):
    mask = gamma_zeros < cutoff
    if np.sum(mask) > 0:
        n_max = np.sum(mask)
        mean_err = np.mean(abs_errors[mask])
        max_err = np.max(abs_errors[mask])
        print(f"{cutoff:>10} {name:>12} {'1-'+str(n_max):>12} {mean_err:>9.2f}% {max_err:>9.2f}%")

# Find optimal cutoff
def mean_error_below_cutoff(cutoff):
    mask = gamma_zeros < cutoff
    if np.sum(mask) < 5:
        return 100
    return np.mean(abs_errors[mask])

# Scan cutoffs
test_cutoffs = np.linspace(30, 200, 100)
test_errors = [mean_error_below_cutoff(c) for c in test_cutoffs]

# Best cutoff for sub-3% error
best_idx = np.argmax(np.array(test_errors) < 3)
if test_errors[best_idx] < 3:
    max_cutoff_3pct = test_cutoffs[np.where(np.array(test_errors) < 3)[0][-1]]
    print(f"\nMax cutoff for <3% mean error: γₙ < {max_cutoff_3pct:.1f}")
    print(f"This corresponds to ~{np.sum(gamma_zeros < max_cutoff_3pct)} zeros")

In [ ]:
print("\n" + "="*70)
print("TEST 3: H* AS NATURAL BOUNDARY")
print("="*70)

# Compare H* cutoff with other GIFT-motivated cutoffs
gift_cutoffs = {
    'b₂ = 21': B2,
    'b₃ = 77': B3,
    'H* = 99': H_STAR,
    'dim(E₇) = 133': DIM_E7,
    'dim(E₈) = 248': DIM_E8,
}

print(f"\n{'Cutoff':>20} {'Value':>8} {'n zeros':>10} {'Mean err':>10} {'Quality':>10}")
print("-" * 65)

for name, cutoff in gift_cutoffs.items():
    mask = gamma_zeros < cutoff
    n_zeros = np.sum(mask)
    if n_zeros > 0:
        mean_err = np.mean(abs_errors[mask])
        quality = "★★★" if mean_err < 3 else "★★" if mean_err < 5 else "★" if mean_err < 10 else ""
        print(f"{name:>20} {cutoff:>8} {n_zeros:>10} {mean_err:>9.2f}% {quality:>10}")

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Predictions with H* boundary
ax1 = axes[0, 0]
ax1.plot(n_values, gamma_zeros, 'ko-', markersize=4, label='Riemann zeros')
ax1.plot(n_values, gamma_pred, 'g--', linewidth=2, label='GIFT: H*/8 × n^(φ-1)')
ax1.axhline(y=H_STAR, color='red', linestyle='-', linewidth=2, label=f'H* = {H_STAR}')
ax1.axhline(y=B3, color='orange', linestyle='--', alpha=0.7, label=f'b₃ = {B3}')
ax1.axhline(y=DIM_E7, color='purple', linestyle=':', alpha=0.7, label=f'dim(E₇) = {DIM_E7}')
ax1.fill_between(n_values, 0, H_STAR, alpha=0.1, color='green', label='Sub-H* regime')
ax1.set_xlabel('n')
ax1.set_ylabel('γₙ')
ax1.set_title('Riemann Zeros with GIFT Boundaries')
ax1.legend(loc='upper left')
ax1.set_ylim(0, 250)
ax1.grid(True, alpha=0.3)

# Plot 2: Error vs γₙ/H*
ax2 = axes[0, 1]
colors = ['green' if g < H_STAR else 'red' for g in gamma_zeros]
ax2.scatter(gamma_ratio, abs_errors, c=colors, alpha=0.7, s=50)
ax2.axvline(x=1, color='red', linestyle='-', linewidth=2, label='γₙ = H*')
ax2.axhline(y=5, color='orange', linestyle='--', label='5% threshold')
ax2.set_xlabel('γₙ / H*')
ax2.set_ylabel('|Error| %')
ax2.set_title('Error vs Normalized Zero Value')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Cumulative mean error
ax3 = axes[1, 0]
cumulative_err = [np.mean(abs_errors[:i+1]) for i in range(len(abs_errors))]
ax3.plot(n_values, cumulative_err, 'b-', linewidth=2)
ax3.axvline(x=n_transition, color='red', linestyle='-', linewidth=2, 
            label=f'n={n_transition} (γₙ crosses H*)')
ax3.axhline(y=3, color='green', linestyle='--', label='3% target')
ax3.axhline(y=5, color='orange', linestyle='--', label='5% threshold')
ax3.set_xlabel('n (including zeros 1 to n)')
ax3.set_ylabel('Cumulative Mean Error %')
ax3.set_title('Error Accumulation vs Zero Count')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Error by regime
ax4 = axes[1, 1]
regimes = ['γₙ < b₃\n(n≤20)', 'b₃ < γₙ < H*\n(n=21-29)', 'H* < γₙ < E₇\n(n=30-45)', 'γₙ > E₇\n(n>45)']
regime_errors = [
    np.mean(abs_errors[gamma_zeros < B3]),
    np.mean(abs_errors[(gamma_zeros >= B3) & (gamma_zeros < H_STAR)]),
    np.mean(abs_errors[(gamma_zeros >= H_STAR) & (gamma_zeros < DIM_E7)]),
    np.mean(abs_errors[gamma_zeros >= DIM_E7])
]
colors = ['green', 'lightgreen', 'orange', 'red']
bars = ax4.bar(regimes, regime_errors, color=colors, alpha=0.7, edgecolor='black')
ax4.axhline(y=5, color='red', linestyle='--', label='5% threshold')
ax4.set_ylabel('Mean Error %')
ax4.set_title('Error by γₙ Regime')
for bar, err in zip(bars, regime_errors):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{err:.1f}%', ha='center', fontsize=10, fontweight='bold')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('K7_Riemann_v6_cutoff_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
print("\n" + "="*70)
print("CONCLUSION")
print("="*70)

# Final statistics
sub_h_mask = gamma_zeros < H_STAR
n_sub_h = np.sum(sub_h_mask)

print(f"""
HYPOTHESIS: The GIFT formula works in the sub-cohomological regime (γₙ < H*).

EVIDENCE:

1. TRANSITION POINT
   - γ₂₉ = {gamma_zeros[28]:.2f} ≈ H* = 99
   - The 29th zero marks the boundary

2. ERROR BY REGIME
   - γₙ < H* (n ≤ {n_sub_h}):  Mean error = {np.mean(abs_errors[sub_h_mask]):.2f}%
   - γₙ > H* (n > {n_sub_h}):  Mean error = {np.mean(abs_errors[~sub_h_mask]):.2f}%
   - Error ratio: {np.mean(abs_errors[~sub_h_mask])/np.mean(abs_errors[sub_h_mask]):.1f}x worse above H*

3. PHYSICAL INTERPRETATION
   - H* = b₂ + b₃ + 1 = 99 encodes total cohomological information
   - Below H*: K₇ topology constrains Riemann zeros
   - Above H*: Asymptotic regime (2πn/ln(n)) takes over

4. REFINED CLAIM
   The GIFT formula γₙ = (H*/8) × n^(φ-1) is valid for:
   
   γₙ < H* = 99  (equivalently, n ≲ 29)
   
   with {np.mean(abs_errors[sub_h_mask]):.2f}% mean accuracy.
""")

# Save results
import json
results = {
    'version': '6.0',
    'hypothesis': 'H* cutoff',
    'cutoff_value': int(H_STAR),
    'n_zeros_below_cutoff': int(n_sub_h),
    'transition_zero': {'n': 29, 'gamma': float(gamma_zeros[28])},
    'error_sub_H_star': float(np.mean(abs_errors[sub_h_mask])),
    'error_super_H_star': float(np.mean(abs_errors[~sub_h_mask])),
    'error_ratio': float(np.mean(abs_errors[~sub_h_mask])/np.mean(abs_errors[sub_h_mask])),
    'conclusion': 'H* = 99 defines the natural boundary for GIFT-Riemann correspondence'
}

with open('K7_Riemann_v6_cutoff_results.json', 'w') as f:
    json.dump(results, f, indent=2)
    
print("\nResults saved to K7_Riemann_v6_cutoff_results.json")